In [30]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import time 

In [31]:
# Set Executable Path & Initialize Chrome Browser
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [4]:
# this dictionary will hold everything we pull from all the sites
scraped_data = {}

In [5]:
# site 1 -
news_url = "https://mars.nasa.gov/news/" # probably need to replace this since it redirects

# use beautiful soup to parse the url above
browser.visit(news_url)
time.sleep(1)
mars_web_html = browser.html
mars_web_soup = BeautifulSoup(mars_web_html, 'html.parser')

news_list = mars_web_soup.find('ul', class_ = 'item_list')
headline_title = news_list.find('li', class_ = 'slide')
mars_headline = headline_title.find('div', class_ = 'content_title').text
mars_body = headline_title.find('div', class_ = 'article_teaser_body' ).text
scraped_data['mars_headline'] = mars_headline
scraped_data['mars_body'] = mars_body
print(mars_headline)
print(mars_body)
print(scraped_data)

NASA's Curiosity Rover Finds an Ancient Oasis on Mars
New evidence suggests salty, shallow ponds once dotted a Martian crater — a sign of the planet's drying climate.
{'mars_headline': "NASA's Curiosity Rover Finds an Ancient Oasis on Mars", 'mars_body': "New evidence suggests salty, shallow ponds once dotted a Martian crater — a sign of the planet's drying climate."}


In [38]:
# site 2 - 
feature_img_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

# use splinter to connect to the url and navigate, then use bs4 to repeat what you did in site 1
browser.visit(feature_img_url)
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
# expand = browser.find_by_css('a.fancybox-expand')
# expand.click()
# time.sleep(1)

feature_img_html = browser.html
feature_img_soup = BeautifulSoup(feature_img_html, 'html.parser')

feature_img = feature_img_soup.find('img', class_ = 'fancybox-image')['src']
feature_img_path = f'https://www.jpl.nasa.gov{feature_img}'

scraped_data['feature_img_path'] = feature_img_path


print(feature_img_path)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17661_ip.jpg


In [39]:
# site 3 - 
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'

# grab the latest tweet and be careful its a weather tweet
browser.visit(mars_weather_url)
time.sleep(1)
mars_weather_html = browser.html
mars_weather_soup = BeautifulSoup(mars_weather_html, 'html.parser')

mars_weather_tweet = mars_weather_soup.find('ol', class_ ='stream-items')
mars_weather = mars_weather_tweet.find('p', class_ = 'tweet-text').text

scraped_data['mars_weather'] = mars_weather

print(mars_weather)

InSight sol 309 (2019-10-10) low -102.3ºC (-152.1ºF) high -26.2ºC (-15.1ºF)
winds from the SSE at 6.1 m/s (13.6 mph) gusting to 18.9 m/s (42.4 mph)
pressure at 7.20 hPapic.twitter.com/sSOjseIl81


In [40]:
# site 4 - 
facts_url = 'https://space-facts.com/mars/'

facts_df = pd.read_html(facts_url)[1]
mars_df = facts_df
mars_df.columns = ['Property', 'Value']

scraped_data['mars_df'] = mars_df

mars_df

,Property,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [41]:
# site 5 
mars_pics_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
# use bs4 to scrape the title and url and add to dictionary
browser.visit(mars_pics_url)
hemisphere_list = []
links = browser.find_by_css("a.product-item h3")
for h in range(len(links)):
   hemisphere = {}
   browser.find_by_css("a.product-item h3")[h].click()
   element = browser.find_link_by_text("Sample").first
   hemisphere["img_url"]=element["href"]
   hemisphere["title"]=browser.find_by_css("h2.title").text
   hemisphere_list.append(hemisphere)
   browser.back()

scraped_data['hemisphere_list'] = hemisphere_list

hemisphere_list

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [42]:
scraped_data

{'mars_headline': "NASA's Curiosity Rover Finds an Ancient Oasis on Mars",
 'mars_body': "New evidence suggests salty, shallow ponds once dotted a Martian crater — a sign of the planet's drying climate.",
 'feature_img_path': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17661_ip.jpg',
 'mars_weather': 'InSight sol 309 (2019-10-10) low -102.3ºC (-152.1ºF) high -26.2ºC (-15.1ºF)\nwinds from the SSE at 6.1 m/s (13.6 mph) gusting to 18.9 m/s (42.4 mph)\npressure at 7.20 hPapic.twitter.com/sSOjseIl81',
 'mars_df':                Property                          Value
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record: